### Passos:
1. Criar um ambiente virtual python:
    - <b>python -m venv nome_do_ambiente</b>
    >obs.: Substitua "nome_do_ambiente" pelo nome que deseja dar ao seu ambiente virtual.
2. Ativar o ambiente virtual criado:
    - <b>No Windows: nome_do_ambiente\Scripts\activate</b>
3. Instalar o Selenium:
    - <b>pip install selenium</b>
    >Obs.: Certifique-se de ter o python instalado na sua máquina.
    <br>
    > Vou instalar o dotenv para carregar os dados confidenciais: 
    <br>
    ><b>pip install python-dotenv</b>
4. Obter o driver do navegador, no caso, iremos utilizar o Chrome.

## No arquivo Python:

1. Importo as bibliotecas utilizadas, no nosso caso o Selenium;

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from dotenv import load_dotenv
import os
import base64

load_dotenv()
num_cartao = os.getenv('NUMERO_CARTAO')
num_matricula = os.getenv('MATRICULA')
qtd_creditos = os.getenv('QUANTIDADE_CREDITOS')

2. Indico o driver do chrome que será utilizado, e a configuração inicial para abrir a janela maximizada;

In [ ]:
chromedriver_exe = r'chromedriver.exe'
options = Options()
options.add_argument("start-maximized")

3. Abro o navegador e vou chamar o site que desejo fazer o teste;

In [ ]:
driver = webdriver.Chrome(service=Service(chromedriver_exe), options=options)
driver.get('https://si3.ufc.br/public//jsp/restaurante_universitario/consulta_comensal_ru.jsf')

4. Insiro os dados nos inputs e clico no botão "Consultar";

In [ ]:
input_num_cartao = driver.find_element(by=By.XPATH, value='//*[@id="form"]/table/tbody/tr[1]/td/input') 
# input_num_cartao = driver.find_element(by=By.NAME, value='form:j_id_jsp_1091681061_2')
input_num_cartao.send_keys(num_cartao)

In [ ]:
# input_num_matricula = driver.find_element(by=By.XPATH, value='//*[@id="form"]/table/tbody/tr[2]/td/input')
input_num_matricula = driver.find_element(by=By.NAME, value='form:j_id_jsp_1091681061_3')
input_num_matricula.send_keys(num_matricula)

In [ ]:
sleep(5)
botao_consultar = driver.find_element(by=By.XPATH, value='//*[@id="form"]/table/tfoot/tr/td/input')
# botao_consultar = driver.find_element(by=By.NAME, value='form:j_id_jsp_1091681061_4')
botao_consultar.click()

5. Insiro a quantidade de créditos e em seguida seleciona a opção de pix e cartão de crédito;

### <!> Se usar apenas o send_keys, vai inserir concatenando as string e o input é limitado a tamanho 2, não vai inserir corretamente quando quiser inserir mais de 10 créditos.<!>

In [ ]:
input_qtd_creditos = driver.find_element(by=By.ID, value='form:qtdCreditos')
# input_qtd_creditos.send_keys(qtd_creditos)
driver.execute_script(f"arguments[0].value = '{qtd_creditos}';", input_qtd_creditos)

In [ ]:
selec_pix_credito = driver.find_element(by=By.ID, value='form:j_id_jsp_540432864_5:0')
selec_pix_credito.click()

###### Se o elemento do botão de pagar não existir, vai gerar o erro de botão não interativo, por isso é ideal deixar um tempo limite para encontrar o elemento na página

In [ ]:
botao_pagar = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, "form:btPagTesouro")))
botao_pagar.click()

In [ ]:
botao_confirmar_pagamento = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, "modalForm2:btConfirmarPagtesouro")))
botao_confirmar_pagamento.click()

6. Irei alternar para o iFlame de pagamento e encontrar os elementos para interagir;

### <!>Se não alternar para o iFlame do pagamento, não encontra os elementos na página principal.<!>

In [ ]:
# Encontre o iframe pelo nome, índice ou elemento
iframe = driver.find_element(by=By.ID, value='iFrameResizer0')
# Alternar para o iframe
driver.switch_to.frame(iframe)

In [ ]:
selec_pix = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div/div[3]/div[2]/div[1]/div[3]/div[1]')
selec_pix.click()

In [ ]:
botao_pagar = driver.find_element(by=By.ID, value='btnPgto')
botao_pagar.click()

7. Por fim, vou retornar o código do pix e o QR code em arquivos.

In [ ]:
qr_code = driver.find_element(by=By.XPATH, value='//img[@class="qr-code-img"]')
src_qr_code = qr_code.get_attribute('src')
codigo_pix = driver.find_element(by=By.XPATH,value='//*[@id="app"]/div/div[5]/div[2]/div/div[2]/div/div[2]/div/div/div[2]/div/p').text

In [ ]:
base = src_qr_code.replace("data:image/png;base64,","")

# Decodifique a string base64 para bytes
imagem_bytes = base64.b64decode(base)

# Salve os bytes da imagem em um arquivo PNG
with open("qr_code.png", "wb") as arquivo:
    arquivo.write(imagem_bytes)

print("Imagem salva com sucesso como 'qr_code.png'")

# Salva o código do pix em outro arquivo txt
with open("codigo_pix.txt","w") as arquivo:
    arquivo.write(codigo_pix)

print(f"Código do Pix salvo em 'codigo_pix.txt'")

In [ ]:
# Para sair do iframe e voltar para o contexto padrão da página principal
driver.switch_to.default_content()
driver.quit()